# Google Sheets Automation with Python

This project demonstrates how to automate data processing and calculations in a Google Sheet using the Google Sheets API and Python. Sensitive information, such as `SPREADSHEET_ID` and `Token`, has been removed for security purposes. You will need to generate your own credentials and obtain these details to run the script.

## Key Features
- **Google Sheets Integration**:
  - Reads data from columns A and B of a specified Google Sheet.
  - Performs row-by-row calculations (sum of two columns).
  - Updates the Google Sheet with results in column C and marks tasks as "Done" in column D.
- **Authentication**:
  - Uses OAuth 2.0 for secure and authorized access to Google Sheets.
  - Stores tokens for seamless re-authentication in future runs.
- **Execution Logging**:
  - Prints detailed logs of the row-by-row processing.
  - Displays the total execution time.

## Why OAuth 2.0?
OAuth 2.0 is a secure industry-standard protocol for authorization that enables applications to access user resources without exposing their credentials.
### **Advantages**:
1. **Enhanced Security**: No need to share sensitive credentials like passwords with third-party applications.
2. **Scoped Access**: Grants limited permissions (e.g., read-only or edit-specific data) for better control.
3. **Token-Based**: Uses tokens that can be refreshed or revoked, reducing the risk of long-term exposure.

In this project, OAuth 2.0 ensures that the application securely accesses and updates Google Sheets on behalf of the user.

## Important Note
- The following sensitive details have been omitted:
  - **SPREADSHEET_ID**: This identifies the specific Google Sheet to be automated.
  - **`token.json` and `Credential.json`**: Used for authentication and authorization.
- **How to Set Up**:
  - Obtain the `SPREADSHEET_ID` from your Google Sheet URL.
  - Generate `Credential.json` by enabling the Google Sheets API in the Google Cloud Console and downloading the credentials.
  - The script will automatically create `token.json` after the first authentication.

## Prerequisites
- Enable the Google Sheets API in your Google Cloud Console.
- Install required Python libraries:
  ```bash
  pip install google-auth google-auth-oauthlib google-api-python-client


In [ ]:
import os.path
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError
import time


# Step 1: Defining the scopes required for accessing Google Sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = "..."

# Authentication and main process
def main():
    credentials = None

    # Step 2: Check if token.json exists
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json", SCOPES)

    # Step 3: If no valid credentials, authenticate the user
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            # Load client secrets from credentials.json and authenticate
            flow = InstalledAppFlow.from_client_secrets_file("Credential.json", SCOPES)
            credentials = flow.run_local_server(port=0)

        # Saving the credentials for next time
        with open("token.json", "w") as token:
            token.write(credentials.to_json())

    try:
        # Step 4: Building the Google Sheets API service
        service = build("sheets", "v4", credentials=credentials)
        sheets = service.spreadsheets()

        # Step 5: Accessing and read data from the spreadsheet
        result = sheets.values().get(spreadsheetId=SPREADSHEET_ID, range="Sheet1!A1:C7").execute()
        values = result.get("values", [])

        # Start timer
        start_time = time.time()

        # Step 6: Processing and print the values
        for row in range(2,9):
            num1 = int(sheets.values().get(spreadsheetId=SPREADSHEET_ID, range=f"sheet1!A{row}").execute().get("values")[0][0])
            num2 = int(sheets.values().get(spreadsheetId=SPREADSHEET_ID, range=f"sheet1!B{row}").execute().get("values")[0][0])
            calculation_result = num1 + num2
            print(f"processing {num1} + {num2}")

            sheets.values().update(spreadsheetId=SPREADSHEET_ID, range=f"sheet1!C{row}",
                                   valueInputOption="USER_ENTERED",
                                   body={"values":[[f"{calculation_result}"]]}).execute()

            sheets.values().update(spreadsheetId=SPREADSHEET_ID, range=f"sheet1!D{row}",
                                   valueInputOption="USER_ENTERED",
                                   body={"values": [["Done"]]}).execute()
        values = result.get("values", [])

        # Printing execution time in one line
        print(f"All calculations completed in {time.time() - start_time:.2f} seconds.")

    # Handle HTTPError exceptions properly
    except HttpError as error:
        print(f"An error occurred: {error}")

# Entry point
if __name__ == "__main__":
    main()

